1. 문서의 내용을 읽는다
2. 문서를 쪼갠다
    - 토큰 수 초과로 답변을 생성하지 못할 수도 있고
    - 문서가 길면 (인풋이 길면) 답변 생성이 오래 걸림
3. 문서 임베딩 후 -> 벡터 데이터베이스에 저장
4. 질문이 있을 때, 벡터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

In [1]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

spltr=RecursiveCharacterTextSplitter(
    chunk_size=1500, # 문서를 쪼갤 때 하나의 청크가 가질 수 있는 토큰 수
    chunk_overlap=200, # 문서를 자를 때 어느정도 겹치게 할 지 (앞뒤 문맥을 같이 전달해주기 위함)
)

loader=Docx2txtLoader('./tax.docx')
doc_list=loader.load_and_split(text_splitter=spltr)

In [2]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv() # 임베딩 과정에도 환경변수 필요하므로 불러와줘야함

embedding=OpenAIEmbeddings(model='text-embedding-3-large')

In [ ]:
import os
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

index_name='tax-index'
pinecone_api_key = os.environ.get("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)

database=PineconeVectorStore.from_documents(doc_list,embedding,index_name=index_name)

c:\Users\User\Desktop\rag-practice\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
query='연봉 5000만원인 거주자의 소득세는 얼마인가요?'
retrieved_docs=database.similarity_search(query, k=3)

In [10]:
from langchain_openai import ChatOpenAI

llm=ChatOpenAI(model='gpt-4o')

In [11]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system",
     "당신은 최고의 한국 소득세 전문가입니다."
     "Context를 참고해서 사용자의 질문에 답변해주세요."),
    ("user",
     "문서(context):\n{context}\n\n"
     "질문(question): {question}\n\n"
     "답변:")
])

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

retriever = database.as_retriever(search_kwargs={"k": 3})

def format_docs(docs):
    return "\n\n".join(d.page_content for d in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [13]:
ai_msg=rag_chain.invoke(query)
ai_msg

'연봉 5,000만원인 거주자의 소득세를 계산하기 위해서는 제55조의 종합소득과세표준에 대한 세율을 적용해야 합니다.\n\n1. 연봉 5,000만원의 경우, 종합소득 과세표준 구간은 "1,400만원 초과 ~ 5,000만원 이하"에 해당합니다.\n2. 해당 구간의 세율은 84만원 + (1,400만원을 초과하는 금액의 15퍼센트)입니다.\n3. 5,000만원에서 1,400만원을 초과한 금액은 3,600만원입니다 (5,000만원 - 1,400만원).\n4. 3,600만원의 15%는 540만원입니다.\n\n따라서, 소득세는 84만원 + 540만원 = 624만원이 됩니다.'